In [2]:
import pandas as pd
import math
import numpy as np
from phase1 import *
from main import *

documents= stemmed_doc
all_words=[]

for doc in documents:
    for word in doc:
        all_words.append(word)

# print(dict.fromkeys(all_words,0))

def get_term_freq(doc):
    words_found=dict.fromkeys(all_words,0)
    for word in doc:
        words_found[word] +=1
    return words_found

term_freq= pd.DataFrame(get_term_freq(documents[0]).values(),index=get_term_freq(documents[0]).keys())


for i in range(1,len(documents)):
    term_freq[i]= get_term_freq(documents[i]).values()

# print(term_freq)

term_freq.columns=['doc'+ str(i) for i in range(1,11)]
print(term_freq)


def get_weighted_term_freq(x):
    if x>0:
        return math.log(x)+1
    return 0

for i in range (1,len(documents)+1):
    term_freq['doc'+str(i)]=term_freq['doc'+str(i)].apply(get_weighted_term_freq)

print(term_freq)
# idf is log(doc num/ term freq in doc)
tfd=pd.DataFrame(columns=['freq','idf'])

for i in range(len(term_freq)):

    frequency=term_freq.iloc[i].values.sum()

    tfd.loc[i ,'freq']= frequency

    tfd.loc[i ,'idf']=math.log10(10 / float((frequency)) )

tfd.index= term_freq.index
print(tfd)    


##### TF IDF #####

term_freq_inverse_doc_freq= term_freq.multiply(tfd['idf'], axis=0)

print(term_freq_inverse_doc_freq)


####### doc length

document_length=pd.DataFrame()

def get_docs_length(col):
    return np.sqrt(term_freq_inverse_doc_freq[col].apply(lambda x : x**2).sum())

for column in term_freq_inverse_doc_freq.columns:
    document_length.loc[0 , column+'_len']= get_docs_length(column)

print(document_length)


#### normalized TF IDF ######


normalized_term_freq_idf= pd.DataFrame()

def get_normalized(col , x):
    try:
        return x / document_length[col+'_len'].values[0]
    
    except:
        return 0
    
for column in term_freq_inverse_doc_freq.columns:
    normalized_term_freq_idf[column]= term_freq_inverse_doc_freq[column].apply(lambda x : get_normalized(column , x))

normalized_term_freq_idf

[['antony', 'brutus', 'caeser', 'cleopatra', 'mercy', 'worser'], ['antony', 'brutus', 'caeser', 'calpurnia'], ['mercy', 'worser'], ['brutus', 'caeser', 'mercy', 'worser'], ['caeser', 'mercy', 'worser'], ['antony', 'caeser', 'mercy'], ['angels', 'fools', 'fear', 'in', 'rush', 'to', 'tread', 'where'], ['angels', 'fools', 'fear', 'in', 'rush', 'to', 'tread', 'where'], ['angels', 'fools', 'in', 'rush', 'to', 'tread', 'where'], ['fools', 'fear', 'in', 'rush', 'to', 'tread', 'where']]
[['antoni', 'brutu', 'caeser', 'cleopatra', 'merci', 'worser'], ['antoni', 'brutu', 'caeser', 'calpurnia'], ['merci', 'worser'], ['brutu', 'caeser', 'merci', 'worser'], ['caeser', 'merci', 'worser'], ['antoni', 'caeser', 'merci'], ['angel', 'fool', 'fear', 'in', 'rush', 'to', 'tread', 'where'], ['angel', 'fool', 'fear', 'in', 'rush', 'to', 'tread', 'where'], ['angel', 'fool', 'in', 'rush', 'to', 'tread', 'where'], ['fool', 'fear', 'in', 'rush', 'to', 'tread', 'where']]
Document 1: ['antoni', 'brutu', 'caeser', 

ZeroDivisionError: float division by zero

In [8]:
q= 'antony AND brutus'
# def phrse_query(q):
# porter = PorterStemmer()
# query_tokens = word_tokenize(q)
# stemmed_query = [porter.stem(token) for token in query_tokens]
# q= ' '.join(stemmed_query)
query_boolean(q)


NameError: name 'normalized_term_freq_idf' is not defined

In [9]:
query = pd.DataFrame(index=normalized_term_freq_idf.index)
query['tf']=[1 if x in q.split() else 0 for x in list(normalized_term_freq_idf.index)]
query['w_tf']=query['tf'].apply(lambda x : get_weighted_term_freq(x))
product = normalized_term_freq_idf.multiply(query['w_tf'], axis=0)
query['idf']=tfd['idf'] * query['w_tf']
query['tf_idf']=query['w_tf'] * query['idf']
query['norm']=0
for i in range(len(query['norm'])):
   query['norm'].iloc[i]= float(query['idf'].iloc[i]) / math.sqrt(sum(query['idf'].values**2))

product2 = product.multiply(query['norm'],axis=0)
scores = {}
for col in product2.columns:
  if 0 in product2[col].loc[q.split()].values:
    pass
  else:
    scores[col] = product2[col].sum()

print(query.loc[q.split()])
print("----------------------------prod-😍----------------------------------")
print(product2[list(scores.keys())].loc[q.split()])
print("----------------------------prod_res.sum()😍----------------------------------")
print(prod_res.sum())
#sum
scores = {}
for col in product2.columns:
  if 0 in product2[col].loc[q.split()].values:
    pass
  else:
    scores[col] = product2[col].sum()
print("----------------------------sim_score-😍----------------------------------")
print("\n")
print(scores ,"\n")
print("\n")

#query length
print("----------------------------DOC_LEN-😍----------------------------------")

print(math.sqrt(sum([x**2 for x in query['idf'].loc[q.split()]])))
print("\n")

#returned
final_score = sorted(scores.items(), key=lambda x: x[1] , reverse=True)
print("----------------------------returned docs-😍----------------------------------")
for doc in final_score:

  print("\n")

  print(doc[0],end='')
  print("\n")

NameError: name 'normalized_term_freq_idf' is not defined

In [6]:
#  AND , OR , NOT اشتغل 

from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

# Instantiate Porter Stemmer
porter = PorterStemmer()

def preprocess_query(query):
    # Tokenize the query and stem each term
    token_docs = word_tokenize(query)
    prepared_doc = [porter.stem(term) for term in token_docs]
    return prepared_doc

def put_query(query, positional_index):
    boolean_operators = ['AND', 'OR', 'NOT']
    boolean_operator = None

    # Check if any boolean operator is present in the query
    for operator in boolean_operators:
        if operator in query:
            boolean_operator = operator
            break

    # If no boolean operator is present, treat the entire query as a phrase
    if boolean_operator is None:
        preprocessed_query = preprocess_query(query)
        positions = find_positions(preprocessed_query, positional_index)
    else:
        # Split the query based on the boolean operator
        query_parts = query.split(boolean_operator)
        preprocessed_query1 = preprocess_query(query_parts[0].strip())
        preprocessed_query2 = preprocess_query(query_parts[1].strip())

        if boolean_operator == 'AND':
            positions = perform_and_operation(preprocessed_query1, preprocessed_query2, positional_index)
        elif boolean_operator == 'OR':
            positions = perform_or_operation(preprocessed_query1, preprocessed_query2, positional_index)
        elif boolean_operator == 'NOT':
            positions = perform_not_operation(preprocessed_query1, preprocessed_query2, positional_index)

    return positions

def find_positions(preprocessed_query, positional_index):
    lis = [[] for i in range(10)]  
    for term in preprocessed_query:
        if term in positional_index.keys():
            for key in positional_index[term][1].keys():
                if lis[key] != []:
                    if lis[key][-1] == positional_index[term][1][key][0] - 1:
                        lis[key].append(positional_index[term][1][key][0])
                else:
                    lis[key].append(positional_index[term][1][key][0])

    positions = []
    for pos, lst in enumerate(lis, start=1):
        if len(lst) == len(preprocessed_query):
            positions.append('document ' + str(pos))
    return positions

def perform_and_operation(query1, query2, positional_index):
    positions1 = find_positions(query1, positional_index)
    positions2 = find_positions(query2, positional_index)
    return set(positions1) & set(positions2)

def perform_or_operation(query1, query2, positional_index):
    positions1 = find_positions(query1, positional_index)
    positions2 = find_positions(query2, positional_index)
    return set(positions1) | set(positions2)

def perform_not_operation(query1, query2, positional_index):
    positions1 = find_positions(query1, positional_index)
    positions2 = find_positions(query2, positional_index)
    return set(positions1) - set(positions2)


user_query = input("Enter your boolean query: ")
preprocessed_query = preprocess_query(user_query)
print('Original Query:', user_query)
print('Preprocessed Query:', preprocessed_query)
result = put_query(user_query, positional_index)
print('Matched Documents:', result)

Original Query: antony OR brutus
Preprocessed Query: ['antoni', 'or', 'brutu']


AttributeError: 'function' object has no attribute 'keys'

In [7]:
from sklearn.preprocessing import normalize

# Assume normalized_term_freq_idf, tfd, and other required functions are defined

def query_boolean(q):
    porter = PorterStemmer()
    query_tokens = word_tokenize(q)
    stemmed_query = [porter.stem(token) for token in query_tokens]

    # Check for boolean operators
    if ' AND ' in q:
        operator = 'AND'
        query_terms = [term.strip() for term in q.split('AND')]
    elif ' OR ' in q:
        operator = 'OR'
        query_terms = [term.strip() for term in q.split('OR')]
    else:
        operator = 'AND'  # Default to OR if no explicit operator is provided
        query_terms = [term.strip() for term in q.split()]

    # Initialize an empty dataframe
    query_result = pd.DataFrame(index=normalized_term_freq_idf.index)

    for term in query_terms:
        # Stem each term
        term_tokens = word_tokenize(term)
        stemmed_term = [porter.stem(token) for token in term_tokens]

        # Create a query vector for the current term
        query = pd.DataFrame(index=normalized_term_freq_idf.index)
        query['tf'] = [1 if x in stemmed_term else 0 for x in normalized_term_freq_idf.index]
        query['w_tf'] = query['tf'].apply(lambda x: get_weighted_term_freq(x))
        query['idf'] = tfd['idf'] * query['w_tf']
        query['tf_idf'] = query['idf'] * query['w_tf']
        query['norm'] = normalize(query[['idf']], norm='l2', axis=0)

        # Perform AND or OR operation
        if operator == 'AND':
            query_result = pd.concat([query_result, query], axis=1, join='inner')  # Use inner join for AND
        elif operator == 'OR':
            query_result = pd.concat([query_result, query], axis=1, join='outer')

    # Return the final result after AND or OR operation
    return query_result


In [5]:
# def query_norm_idf(q):
    # phrse_query('fools fear')
porter = PorterStemmer()
query_tokens = word_tokenize(q)
stemmed_query = [porter.stem(token) for token in query_tokens]
q= ' '.join(stemmed_query)
query = pd.DataFrame(index=normalized_term_freq_idf.index)
query['tf']=[1 if x in q.split() else 0 for x in normalized_term_freq_idf.index]
query['w_tf']= query['tf'].apply(lambda x: get_weighted_term_freq(x))
product = normalized_term_freq_idf.multiply(query['w_tf'], axis= 0)
            # product
query['idf'] = tfd['idf'] * query['w_tf']
query['tf_idf'] = query['idf'] * query['w_tf']
query['norm']=0
for i in range(len(query)):
    query['norm'].iloc[i]= float(query['idf'].iloc[i]) / math.sqrt(sum(query['idf'].values**2))
product2=product.multiply(query['norm'], axis=0)
    # product2
query


C:\Users\H P\AppData\Local\Temp\ipykernel_4120\1056851837.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  query['norm'].iloc[i]= float(query['idf'].iloc[i]) / math.sqrt(sum(query['idf'].values**2))
C:\Users\H P\AppData\Local\Temp\ipykernel_4120\1056851837.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.7071067811865475' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  query['norm'].iloc[i]= float(query['idf'].iloc[i]) / math.sqrt(sum(query['idf'].values**2))


,tf,w_tf,idf,tf_idf,norm
antoni,1,1.0,0.522879,0.522879,0.707107
brutu,1,1.0,0.522879,0.522879,0.707107
caeser,0,0.0,0.0,0.0,0.000000
cleopatra,0,0.0,0.0,0.0,0.000000
merci,0,0.0,0.0,0.0,0.000000
worser,0,0.0,0.0,0.0,0.000000
calpurnia,0,0.0,0.0,0.0,0.000000
angel,0,0.0,0.0,0.0,0.000000
fool,0,0.0,0.0,0.0,0.000000
fear,0,0.0,0.0,0.0,0.000000


In [12]:
# # query_norm_idf('fools fear')
# print(query_norm_idf('antony brutus'))

ZeroDivisionError: float division by zero

In [106]:
query_length=math.sqrt(sum([x**2 for x in query['idf'].loc[q.split()]]))
query_length

KeyError: "['and'] not in index"

In [4]:
query_length = 0

# Iterate through the query terms
for term in q.split():
    term_tokens = word_tokenize(term)
    stemmed_term = [porter.stem(token) for token in term_tokens]

    # Check if the term is present in the query vector
    if stemmed_term[0] in query.index:
        query_length += query['idf'].loc[stemmed_term[0]] ** 2

# Take the square root of the sum of squared IDF values
query_length = math.sqrt(query_length)
query_length

TypeError: argument of type 'builtin_function_or_method' is not iterable

In [100]:
scores = {}
for col in product2.columns:
    if 0 in product2[col].loc[q.split()].values:
        pass
    else:
        scores[col]=product2[col].sum()

In [141]:
scores = {}

# Iterate through the columns of the product2 dataframe
for col in product2.columns:
    # Extract the relevant IDF values from the query vector
    query_idf_values = query['idf'].loc[product2.index]

    # Check if any of the relevant IDF values is zero
    if 0 in query_idf_values.values:
        # Skip if any term in the query has IDF of 0
        pass
    else:
        # Calculate the score by summing the product of normalized TF-IDF values and query IDF values
        scores[col] = (product2[col] * query_idf_values).sum()

# Print or use the scores as needed
scores


{}

In [143]:
# scores = {}

# Iterate through the columns of the product2 dataframe
for col in product2.columns:
    # Extract the relevant IDF values from the query vector
    query_idf_values = query['idf'].loc[product2.index]

    # Calculate the score by summing the product of normalized TF-IDF values and query IDF values
    scores[col] = (product2[col] * query_idf_values).sum()

# Filter out documents that don't contain any term from the query
filtered_scores = {key: score for key, score in scores.items() if any(query['tf'].loc[product2.index].astype(bool))}
filtered_scores


{'doc1': 0.2815141484620463,
 'doc2': 0.302159654545043,
 'doc3': 0.0,
 'doc4': 0.24692096410061823,
 'doc5': 0.0,
 'doc6': 0.2867167166390384,
 'doc7': 0.0,
 'doc8': 0.0,
 'doc9': 0.0,
 'doc10': 0.0}

In [144]:
scores

{'doc1': 0.2815141484620463,
 'doc2': 0.302159654545043,
 'doc3': 0.0,
 'doc4': 0.24692096410061823,
 'doc5': 0.0,
 'doc6': 0.2867167166390384,
 'doc7': 0.0,
 'doc8': 0.0,
 'doc9': 0.0,
 'doc10': 0.0}

In [66]:
prod_res= product2[scores.keys()].loc[q.split()]

In [128]:
prod_res = product2[filtered_scores.keys()].loc[query.index.intersection(q.split())]

In [129]:
prod_res

,doc1,doc2,doc3,doc4,doc5,doc6,doc7,doc8,doc9,doc10
antoni,0.269196,0.288939,0.0,0.000000,0.0,0.548343,0.0,0.0,0.0,0.0
brutu,0.269196,0.288939,0.0,0.472234,0.0,0.000000,0.0,0.0,0.0,0.0


In [87]:
prod_sum=prod_res.sum()
prod_sum

doc1     0.538393
doc2     0.577877
doc3     0.000000
doc4     0.472234
doc5     0.000000
doc6     0.548343
doc7     0.000000
doc8     0.000000
doc9     0.000000
doc10    0.000000
dtype: float64

In [88]:
final_score=sorted(prod_sum.items(), key= lambda x :x[1], reverse= True)
final_score

[('doc2', 0.5778771030041435),
 ('doc6', 0.5483426496621455),
 ('doc1', 0.5383927937463102),
 ('doc4', 0.47223369916907476),
 ('doc3', 0.0),
 ('doc5', 0.0),
 ('doc7', 0.0),
 ('doc8', 0.0),
 ('doc9', 0.0),
 ('doc10', 0.0)]

In [89]:
for doc in final_score:
    print(doc[0], end=' ')

doc2 doc6 doc1 doc4 doc3 doc5 doc7 doc8 doc9 doc10 